### **Nev: Veres Melitta-Kinga**

**Azonosito: vmgi4447**

1.Í<span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">rjunk&nbsp;</span>  **tárolt eljárást**<span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">, amely meghatározza azokat a kocsmákat, amelyekben legalább&nbsp;2 olyan&nbsp;italt árulnak, mint a paraméterként megadott kocsma (@KocsmaNev varchar(30) típusú)! Ha nincs ilyen kocsma, írjunk ki hibaüzenetet, majd szúrjuk be a kocsmát az adatbázisba!</span>

In [2]:
CREATE OR ALTER PROCEDURE KocsmaItalok
@KocsmaNev VARCHAR(30)
AS
BEGIN
    IF @KocsmaNev IS NULL
    BEGIN
        PRINT @KocsmaNev + ' nem letezik'
        INSERT INTO Kocsmak(Nev) VALUES (@KocsmaNev)
    END
    ELSE
    BEGIN
        CREATE TABLE #Italnevek(Italok VARCHAR(30))

        INSERT #Italnevek(Italok)
        SELECT i.Nev FROM Kocsmak k 
        LEFT JOIN Arak a ON k.KocsmaID = a.KocsmaID
        LEFT JOIN Italok i ON a.ItalID = i.ItalID
        WHERE k.Nev = @KocsmaNev

        CREATE TABLE #Eredmeny(Nev VARCHAR(30))
        INSERT #Eredmeny(Nev)
        SELECT k.Nev
        FROM Kocsmak k 
        LEFT JOIN Arak a ON k.KocsmaID = a.KocsmaID
        LEFT JOIN Italok i ON a.ItalID = i.ItalID
        WHERE i.Nev IN (SELECT Italok FROM #Italnevek)
        GROUP BY k.Nev
        HAVING COUNT(i.Nev) >= 2
        SELECT * FROM #Eredmeny
    END
END

Commands completed successfully.

Batch execution time: 00:00:00.015

Total execution time: 00:00:00.015

In [2]:
EXEC KocsmaItalok @KocsmaNev = 'Frederico Sawley'

(7 rows affected)

(6 rows affected)

(6 rows affected)

Batch execution time: 00:00:00.119

Total execution time: 00:00:00.119

Nev
Frederico Sawley
Hedwiga Ledbetter
Lewes Ashbrook
Orren Broadhurst
Perrine Chuney
Sandra Tweed


2. <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">Írjunk&nbsp;</span> **függvényt**<span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">, melynek bemenő paraméterei: @pBaratNev, @pItaltipusNev. A függvény egy táblában térítse vissza azon kocsmá(k) nevét, címét és a kapható italok nevét és árát mely(ek)et a paraméterként megadott barát kedvel és ahol a megadott italtípusból kapható ital!</span>

In [3]:
CREATE OR ALTER FUNCTION dbo.Kocsmanevek(@pBaratNev VARCHAR(30), @pItaltipusNev VARCHAR(30))
RETURNS @Ujtabla TABLE(Nev VARCHAR(30), Cim VARCHAR(30), Ital VARCHAR(30), ItalAr INT)
AS
BEGIN
    INSERT INTO @Ujtabla (Nev, Cim, Ital, ItalAr)
    SELECT k.Nev, k.Cim, i.Nev, a.Ar FROM Kocsmak k
    JOIN Arak a ON k.KocsmaID=a.KocsmaID
    JOIN Italok i ON a.ItalID=i.ItalID
    JOIN ItalTipusok it ON i.TipusID=it.TipusID
    JOIN Kedvencek ke ON k.KocsmaID=ke.KocsmaID
    JOIN Baratok b ON ke.BaratID=b.BaratID
    WHERE b.Nev=@pBaratNev AND it.TipusNev=@pItaltipusNev

    RETURN;
END

Commands completed successfully.

Batch execution time: 00:00:00.019

Total execution time: 00:00:00.019

In [4]:
DECLARE @ResultTable TABLE(Nev VARCHAR(30), Cim VARCHAR(30), Ital VARCHAR(30), ItalAr INT);
INSERT INTO @ResultTable
SELECT * FROM dbo.Kocsmanevek('Rik Turbard', 'tonic') /*pirossal alahuzza a dbo.Kocsmanevek-et de attol meg jol fut le*/
SELECT * FROM @ResultTable

(2 rows affected)

(2 rows affected)

Batch execution time: 00:00:00.098

Total execution time: 00:00:00.098

Nev,Cim,Ital,ItalAr
Hedwiga Ledbetter,6 Packers Parkway,Coca Cola,15
Sandra Tweed,8 Superior Alley,Kinley,29


3. <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">Írjunk&nbsp;</span> **DELETE triggert**<span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">, mely akkor aktiválódik, ha törlünk</span> **egy vagy több** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">sort az ItalTipusok táblából! (</span>**Figyelem!** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">A hivatkozási épség megszorítások fenntartása miatt probléma lehet, ha egy olyan italtípust szeretnék törölni, melyhez tartozik ital. Ezt a problémát orvosoljuk a trigger segítségével! Az adott italtípus mindenképp törlődjön a trigger végrehajtásának következtében!)</span>

In [4]:
CREATE TRIGGER dbo.ItalTipusoktorlese ON ItalTipusok
INSTEAD OF
DELETE
AS
BEGIN
    DECLARE @pItalTipusID INT

    SET @pItalTipusID = ( SELECT TipusID FROM Deleted )

    IF (SELECT COUNT(*) FROM Italok 
    JOIN Arak
    ON Italok.ItalID=Arak.ItalID
    WHERE TipusID = @pItalTipusID) > 0
    BEGIN
        DELETE FROM Arak WHERE ItalID IN (SELECT ItalID FROM Italok WHERE TipusID=@pItalTipusID)
        DELETE FROM Italok WHERE TipusID = @pItalTipusID;
    END

    DELETE FROM ItalTipusok WHERE TipusID = @pItalTipusID
END

Commands completed successfully.

Batch execution time: 00:00:00.011

Total execution time: 00:00:00.011

In [31]:
/*tesztelem az dbo.ItalTipusoktorlese triggert*/
DELETE FROM ItalTipusok
WHERE TipusID=3

(0 rows affected)

(0 rows affected)

Batch execution time: 00:00:00.021

Total execution time: 00:00:00.021

4. <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">Írjunk&nbsp;</span> **INSERT** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">és</span> **UPDATE** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">triggert az Arak táblára vonatkozóan, amely csak akkor engedélyezi a művelet végrehajtását, ha az árak összértéke</span> **5** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">és</span> **1000** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">között marad!</span>

In [5]:
CREATE TRIGGER dbo.InsertmegUpdate ON Arak
FOR
INSERT, UPDATE
AS
BEGIN
    DECLARE @pAr REAL

    SET @pAr=(SELECT SUM(Ar) FROM Arak) + (SELECT Ar FROM Inserted)

    IF @pAr BETWEEN 5 AND 1000
    BEGIN
        UPDATE Arak
        SET KocsmaID=Inserted.KocsmaID,
        ItalID=Inserted.ItalID,
        Ar=Inserted.AR
        FROM Arak
        JOIN Inserted 
        ON Arak.ItalID=Inserted.ItalID AND Arak.KocsmaID=Inserted.KocsmaID
    END
    ELSE
    BEGIN
        PRINT 'A muvelet nem vegezheto el'
        ROLLBACK
    END
END

Commands completed successfully.

Batch execution time: 00:00:00.005

Total execution time: 00:00:00.005

In [51]:
/*tesztelem az dbo.InsertmegUpdate triggert*/
INSERT Arak VALUES(1, 7, 725)


A muvelet nem vegezheto el

: Msg 3609, Level 16, State 1, Line 1
The transaction ended in the trigger. The batch has been aborted.

Batch execution time: 00:00:00.013

Total execution time: 00:00:00.013

In [52]:
/*tesztelem az dbo.InsertmegUpdate triggert*/
UPDATE Arak
SET Ar=726
WHERE KocsmaID=1 AND ItalID=6

A muvelet nem vegezheto el

: Msg 3609, Level 16, State 1, Line 1
The transaction ended in the trigger. The batch has been aborted.

Batch execution time: 00:00:00.004

Total execution time: 00:00:00.004

5. <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">Írjunk&nbsp;</span>  **DELETE triggert**<span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">, amely nem enged törölni olyan kocsmát, melyet legalább 3-an kedvelnek és a nyitás dátuma 2015 után történt!</span>

In [6]:
CREATE TRIGGER dbo.Nemtorlunkkocsmat ON Kocsmak
INSTEAD OF
DELETE
AS
BEGIN
    DECLARE @pKocsmaID INT

    SET @pKocsmaID = (SELECT KocsmaID FROM Deleted)

    IF (SELECT YEAR(NyitasDatuma) FROM Deleted) >= 2015 AND
    (SELECT COUNT(BaratID) FROM Kedvencek
    WHERE KocsmaID=@pKocsmaID
    ) >=3 
    BEGIN
        PRINT 'Nem torolheto a kocsma a tablabol.'
        ROLLBACK
    END
    ELSE
    BEGIN
        DELETE FROM Kedvencek
        WHERE KocsmaID=@pKocsmaID

        DELETE FROM Arak
        WHERE KocsmaID=@pKocsmaID

        DELETE FROM Kocsmak
        WHERE KocsmaID = @pKocsmaID;
    END
END

Commands completed successfully.

Batch execution time: 00:00:00.011

Total execution time: 00:00:00.011

In [21]:
/*tesztelem a dbo.Nemtorlunkkocsmat triggert*/
DELETE FROM Kocsmak
WHERE Nev='kocsma'

Nem torolheto a kocsma a tablabol.

: Msg 3609, Level 16, State 1, Line 1
The transaction ended in the trigger. The batch has been aborted.

Batch execution time: 00:00:00.015

Total execution time: 00:00:00.015

6. **a.** <span style="background-color: rgb(255, 255, 255); color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;">Hozzunk létre egy új táblát:</span>

  Arak\_Log(ID, Idopont, Muvelet, KocsmaID, KocsmaNev, ItalID, ItalNev, RegiAr, UjAr),

ahol Idopont-datetime, Muvelet-varchar(50), RegiAr-int, UjAr-int típusú mezők.

In [1]:
CREATE TABLE dbo.Arak_Log(
    ID INT IDENTITY(1,1) PRIMARY KEY,
    Idopont DATETIME,
    Muvelet VARCHAR(50),
    KocsmaID INT FOREIGN KEY REFERENCES Kocsmak(KocsmaID),
    KocsmaNev VARCHAR(30),
    ItalID INT FOREIGN KEY REFERENCES Italok(ItalID),
    ItalNev VARCHAR(30),
    RegiAR INT,
    UjAr INT
)

Commands completed successfully.

Batch execution time: 00:00:00.006

Total execution time: 00:00:00.006

6. <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);"><strong>b.</strong>&nbsp;Oldjuk meg, hogy a&nbsp;</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">Kocsmak táblából való törlés esetén az Arak_Log tábla&nbsp;</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">KocsmaID</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">mezőjének értéke&nbsp;</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">NULL-ra módosuljon!</span>

In [7]:
CREATE TRIGGER dbo.Kocsmaktorleseeseten ON Kocsmak
AFTER
DELETE
AS
BEGIN
    DECLARE @pKocsmaID INT

    SET @pKocsmaID=(SELECT KocsmaID FROM Deleted)

    UPDATE dbo.Arak_Log
    SET KocsmaID = NULL
    WHERE KocsmaID = @pKocsmaID

END

Commands completed successfully.

Batch execution time: 00:00:00.003

Total execution time: 00:00:00.003

In [34]:
/*tesztelem a dbo.Kocsmaktorleseeseten trigget*/
DELETE FROM Kocsmak
WHERE Nev='Sosanna Segeswoeth'

(1 row affected)

(0 rows affected)

(1 row affected)

(1 row affected)

(1 row affected)

Batch execution time: 00:00:00.044

Total execution time: 00:00:00.044

6. <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);"><strong>c.</strong>&nbsp;Írjunk&nbsp;</span> **UPDATE triggert**<span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">, mely az Arak tábla Ar mezőjének módosításakor beszúr egy új sort a Arak_Log táblába. Az időpont legyen a módosítás időpontja, művelet szövege: ‘ar növelése’ vagy ‘ar csökkentése’, RegiAr-régi érték, UjAr-új érték.</span>

In [14]:
CREATE TRIGGER dbo.Armodositas ON Arak
AFTER
UPDATE 
AS
BEGIN
    DECLARE @pRegiAr INT, @pUjAr INT, @pMuvelet VARCHAR(50)

    SET @pRegiAr = (SELECT Ar FROM Deleted)
    SET @pUjAr = (SELECT Ar FROM Inserted)

    IF @pRegiAr <> @pUjAr
    BEGIN
    IF @pRegiAr > @pUjAr
    BEGIN
        SET @pMuvelet='ar csokkentese'
    END
    ELSE
    BEGIN
        SET @pMuvelet='ar novelese'
    END

    INSERT INTO Arak_Log(Idopont, Muvelet, KocsmaID, KocsmaNev, ItalID, ItalNev, RegiAR, UjAr)
    VALUES (GETDATE(), @pMuvelet, (SELECT KocsmaID FROM Deleted), (SELECT Nev FROM Kocsmak WHERE KocsmaID=(SELECT KocsmaID FROM Deleted)), 
    (SELECT ItalID FROM Deleted), (SELECT Nev FROM Italok WHERE ItalID=(SELECT ItalID FROM Deleted)), 
    @pRegiAr, @pUjAr);
    END
END

Commands completed successfully.

Batch execution time: 00:00:00.004

Total execution time: 00:00:00.004

In [13]:
/*tesztelem a dbo.Armodositas triggert*/
UPDATE Arak 
SET Ar=10
WHERE KocsmaID=1 AND ItalID=2

(1 row affected)

(1 row affected)

(1 row affected)

Batch execution time: 00:00:00.004

Total execution time: 00:00:00.004